In [1]:
import time
start_time = time.time()

In [2]:
import pandas as pd
import os
from bidi.algorithm import get_display
import re
import multiprocessing

In [3]:
def send_email(user='dsakaidf@gmail.com', pwd='d54k4idf!', recipient='shkasta@post.bgu.ac.il',
               subject='finish expirement', body='finish the expirement'):
    import smtplib

    gmail_user = user
    gmail_pwd = pwd
    FROM = user
    TO = recipient if type(recipient) is list else [recipient]
    SUBJECT = subject
    TEXT = body

    # Prepare actual message
    message = """From: %s\nTo: %s\nSubject: %s\n\n%s
    """ % (FROM, ", ".join(TO), SUBJECT, TEXT)
    try:
        # SMTP_SSL Example
        server_ssl = smtplib.SMTP_SSL("smtp.gmail.com", 465)
        server_ssl.ehlo()  # optional, called by login()
        server_ssl.login(gmail_user, gmail_pwd)
        # ssl server doesn't support or need tls, so don't call server_ssl.starttls()
        server_ssl.sendmail(FROM, TO, message)
        # server_ssl.quit()
        server_ssl.close()
        print('successfully sent the mail')
    except:
        print("failed to send mail")


In [4]:
import pickle

In [5]:
file_type = 'thread'
field_name = 'title'

In [6]:
pickle_file = open(f'{file_type}_{field_name}_token_lists.pickle', "rb" )
corpus = pickle.load( pickle_file )
pickle_file.close()

In [7]:
##Creating a list of stop words and adding custom stopwords
import codecs
with codecs.open('stopwords.txt', 'r', encoding='utf8') as f:
    text = f.read()
stop_words = set(word for word in text.split('\r\n'))

In [8]:
corpus_temp = list()
for sentence in corpus:
    sentence_temp = list()
    for word in sentence:
        try:
            if len(word) > 1 and not word in stop_words:
                sentence_temp.append(word.replace('.','').replace(',','').replace('?','').replace('!',''))
        except:
            print(word)
    corpus_temp.append(sentence_temp)
corpus = corpus_temp

In [9]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel


# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

In [10]:
data_lemmatized = corpus

In [11]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

id2word.filter_extremes(no_below = 0.05, no_above=0.5)
# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1)]]


In [12]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [13]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.451*"גיוס" + 0.319*"מיון" + 0.046*"קצונה" + 0.046*"סייבר" + 0.027*"תכנות" '
  '+ 0.014*"משפט" + 0.000*"מודיעין" + 0.000*"זימון" + 0.000*"תאריך" + '
  '0.000*"אוויר"'),
 (1,
  '0.542*"תותחן" + 0.000*"חוד" + 0.000*"גנא" + 0.000*"יוקרה" + 0.000*"קצין" + '
  '0.000*"יציאה" + 0.000*"חובש" + 0.000*"קו" + 0.000*"נס" + 0.000*"אוויר"'),
 (2,
  '0.462*"צנחן" + 0.000*"מדים" + 0.000*"תאריך" + 0.000*"קרוב" + 0.000*"פטור" + '
  '0.000*"תג" + 0.000*"אוויר" + 0.000*"דובדבן" + 0.000*"רפ" + 0.000*"סם"'),
 (3,
  '0.298*"סיווג" + 0.208*"ראיון" + 0.000*"פצצה" + 0.000*"מודיעין" + '
  '0.000*"זימון" + 0.000*"עישון" + 0.000*"ערך" + 0.000*"סיכוי" + 0.000*"תחקיר" '
  '+ 0.000*"סם"'),
 (4,
  '0.207*"טופס" + 0.206*"לוחמה" + 0.172*"רופא" + 0.125*"ים" + 0.081*"פנייה" + '
  '0.000*"תקשוב" + 0.000*"גימל" + 0.000*"אוויר" + 0.000*"חיל" + 0.000*"עריק"'),
 (5,
  '0.258*"צבא" + 0.179*"נשק" + 0.041*"מטוס" + 0.000*"בגרות" + 0.000*"אוויר" + '
  '0.000*"איש" + 0.000*"מיעוט" + 0.000*"שמחה" + 0.000*"קטע" + 0.000*"אלוה

In [14]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -22.437067098386976

Coherence Score:  0.6114177041195425


In [15]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.391825  0.238631       1        1  21.139151
19    -0.117963 -0.151847       2        1  11.980206
9     -0.138646 -0.258921       3        1  11.544856
13    -0.105451 -0.114076       4        1   8.528807
4     -0.078718 -0.059707       5        1   6.909637
14    -0.031096 -0.009178       6        1   5.823316
7      0.010360  0.013769       7        1   5.683458
8      0.044734  0.024576       8        1   3.395890
18     0.042238  0.023818       9        1   3.159313
16     0.037905  0.022956      10        1   3.088436
17     0.063400  0.027589      11        1   2.620080
5      0.070463  0.028124      12        1   2.542465
2      0.071847  0.028077      13        1   2.473423
12     0.066852  0.027894      14        1   2.316213
3      0.066581  0.027876      15        1   2.213576
11     0.077374  0.027972      16        1   1.950581
1      0.060454  0.027195      17        1   1.760105
10     0.077470  0.027930      18        1   1.190682
15     0.086438  0.024524      19        1   1.012292
6      0.087586  0.022799      20        1   0.667508, topic_info=            Term         Freq       Total Category  logprob  loglift
5          תפקיד  2690.000000  2690.00000  Default  30.0000   30.000
64          גיוס  3217.000000  3217.00000  Default  29.0000   29.000
71          מיון  2275.000000  2275.00000  Default  28.0000   28.000
41        פרופיל  1552.000000  1552.00000  Default  27.0000   27.000
4          שיבוץ   951.000000   951.00000  Default  26.0000   26.000
...          ...          ...         ...      ...      ...      ...
1451       מקוון     0.051258     2.04521  Topic20  -8.3887    1.323
1450     סיכווים     0.051258     2.04521  Topic20  -8.3887    1.323
1449  עשות(הורים     0.051258     2.04521  Topic20  -8.3887    1.323
1448       נהדסה     0.051258     2.04521  Topic20  -8.3887    1.323
1447      שאלהמה     0.051258     2.04521  Topic20  -8.3887    1.323

[750 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
91       18  0.983355     בדיקה
7        16  0.966695    ביטחון
37        6  0.995576      בסיס
20        2  0.996872     גיבוש
64        1  0.999527      גיוס
28        4  0.998242      חייל
29        8  0.980393     חימוש
24        3  0.992864       חיר
115       5  0.996665      טופס
22       11  0.984710      טייס
86        7  0.996549   טירונות
58       10  0.965034     טלפון
106       3  0.997910     יחידה
79        5  0.994166        ים
21       16  0.982765       כלא
19       11  0.992714      לוחם
33        5  0.995419     לוחמה
51       16  0.985488      מבחן
32       19  0.874371     מדינה
15       12  0.945877      מטוס
71        1  0.998971      מיון
59       10  0.972502     מכינה
107       4  0.993472      מפקד
87        7  0.973612       משא
113       1  0.983549      משפט
96       12  0.985133       נשק
63       15  0.993025     סיווג
72        1  0.995225     סייבר
44        3  0.997397    סיירות
30        8  0.930648       עדה
74        3  0.906249      עונש
108      18  0.970264      עוקץ
60        7  0.997113      עזרה
38        5  0.989329     פנייה
26       10  0.987381  פסיכיאטר
41        3  0.998926    פרופיל
52       12  0.989278       צבא
92        4  0.998148        צו
16       13  0.995818      צנחן
39        8  0.996847        קב
109       1  0.993650     קצונה
31       15  0.984729     ראיון
40        5  0.994577      רופא
118       4  0.980627       רשת
104      14  0.980127      שביל
27       10  0.993672     שחרור
4         6  0.997931     שיבוץ
100       9  0.996964     שירות
93       14  0.993742     שריון
101       8  0.973315       תור
77       17  0.994391     תותחן
84       19  0.939184       תיק
73        1  0.990135     תכנות
5         2  0.999180     תפקיד, R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 20, 10, 14, 5, 15, 8, 9, 19, 

In [16]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topics, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
        print('calc with {num_topics} done')
    return model_list, coherence_values

In [17]:
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=2, limit=500, step=6)

KeyboardInterrupt: 

In [ ]:
##### Show graph
limit=500; start=2; step=6;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [ ]:
# Select the model and print the topics
optimal_model = model_list[3]
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))

In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(optimal_model, corpus, id2word)
vis

send_email()

In [ ]:
elapsed_time = time.time() - start_time
print(elapsed_time)

In [ ]:
from gensim.models import HdpModel
hdp_model = HdpModel(corpus, id2word)

In [ ]:
# Compute Perplexity
#print('\nPerplexity: ', hdp_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=hdp_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
# Print the Keyword in the 10 topics
pprint(hdp_model.print_topics())
doc_lda = hdp_model[corpus]